In [6]:
import sqlite3
import prettytable
import pandas as pd

prettytable.DEFAULT = 'DEFAULT'

In [7]:
import yfinance as yf
from time import sleep

def get_data(con, stocks=[], freq='yearly'):
    
    for stock in stocks:
        try:
            sleep(0.3)
            
            ticker = yf.Ticker(stock)

            income_stmt = ticker.get_income_stmt(freq=freq)
            balance_sheet = ticker.get_balance_sheet(freq=freq)
            cash_flow = ticker.get_cash_flow(freq=freq)

            df = pd.concat(
                [
                    income_stmt, 
                    balance_sheet,
                    cash_flow
                ],
                axis=0, 
                join='inner'
            ).T.sort_index().drop_duplicates(keep='first')

            df.to_sql(f'{stock}_{freq.upper()}', con, if_exists='replace', index='Datetime')
            
            con.commit()
            
        except Exception as e:
            print(f'Error fetching data for {stock}/n {e}')

In [8]:
def get_stocks(file=None):
    
    stocks = []
    
    try:
        with open(file, 'r') as f:
            stocks = [stock.strip() for stock in f.readlines()]
            
    except FileNotFoundError:
        print(f'{file} not found.')
    
    return stocks

In [9]:
def get_tables(cur):
    
    cur.execute('SELECT NAME FROM sqlite_master WHERE TYPE="table"')
    
    print(prettytable.from_db_cursor(cur))

In [ ]:
if __name__ == '__main__':
    
    db_name = 'FINANCIALS'
    file = 'data/watchlist.txt'
    freq = 'quarterly'
    
    con = sqlite3.connect(f'data/{db_name}.db')
    cur = con.cursor()
    
    stocks = get_stocks(file)
    
    get_data(con, stocks, freq)
    get_tables(cur)